# 1_Test_enet_bce

1. Generate metrics: ROC curves, confusion matrix, `metrics_per_class_{train/val/test}.csv`
2. Generate sigmoid points

In [1]:
import argparse
import sys

experiment_name = 'enet_model_bce'
im_size = 300
batch_size = 64
data_path = '/home/sebastian/codes/data/ABGQI_mel_spectrograms/'
file_name = 'test'
resume = f'./{experiment_name}/model_weights.pth'

In [2]:
args = argparse.Namespace()

def get_args_parser():
    parser = argparse.ArgumentParser(description="Train and test a model with folder-based data structure.")
    parser.add_argument('--experiment_name', type=str, default=experiment_name)
    parser.add_argument('--resume', type=str, default=experiment_name)
    parser.add_argument('--im_size', type=tuple, default=im_size)
    parser.add_argument('--batch_size', type=int, default=batch_size)
    parser.add_argument('--data_path', type=str, default=data_path)
    parser.add_argument('--file_name', default=file_name, type=str, help='file_name')
    return parser

def parse_args():
    global args
    parser = get_args_parser()
    if 'ipykernel' in sys.modules:
        args, unknown = parser.parse_known_args(namespace=args)
    else:
        args = parser.parse_args(namespace=args)
    return args

args = parse_args()
print(args)

Namespace(experiment_name='enet_model_bce', resume='enet_model_bce', im_size=300, batch_size=64, data_path='/home/sebastian/codes/data/ABGQI_mel_spectrograms/', file_name='test')


In [3]:
import torch
import os
import argparse
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np
import time
from tqdm import tqdm
from net.metrics import get_metrics, count_parameters
from net.models import *

2024-11-22 15:36:14.555092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 15:36:14.555126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 15:36:14.556246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 15:36:14.561510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 15:36:15.211508: W tensorflow/compiler/tf2

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dir = os.path.join(args.data_path, file_name)
test_transform = transforms.Compose([
        transforms.Resize(args.im_size),
        transforms.CenterCrop(args.im_size),
        transforms.ToTensor()
])
specific_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
specific_dataloader = DataLoader(specific_dataset, batch_size=args.batch_size, shuffle=False)
num_classes = len(specific_dataset.classes)
class_names = specific_dataset.classes
class_names

['anthro', 'bio', 'geo', 'other', 'quiet']

In [8]:
model = load_model(resume, num_classes, device)
model

CustomEfficientNet(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
              (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
          

In [6]:
print(args)

Namespace(experiment_name='enet_model_bce', resume='enet_model_bce', im_size=300, batch_size=64, data_path='/home/sebastian/codes/data/ABGQI_mel_spectrograms/', file_name='test')


In [7]:
metrics_df_test, indv_metrics_df_test  = evaluate_test_bce(specific_dataloader, model, device,class_names, experiment_name, file_name)
print(f"Metrics on test-images during testing:\n{metrics_df_test}")
print(f"Extra Metrics on test-images during testing:\n{indv_metrics_df_test}")
metrics_df_test.to_csv(f'./{experiment_name}/metrics_per_class_{file_name}.csv', index=False)
indv_metrics_df_test.to_csv(f'./{experiment_name}/metrics_indv_extras_{file_name}.csv', index=False)

ValueError: Target size (torch.Size([64])) must be the same as input size (torch.Size([64, 5]))